In [ ]:
import sys
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from copy import copy
from tqdm.auto import tqdm

sys.path.append('../')
from model_comparison import *
import ggseg

In [ ]:
from plotnine import *
import plotnine.options as plotnine_opts

In [ ]:
data = pd.read_parquet('../final-results/raw_results.parquet.gzip').query('val_lower_ci > 0')

# we only need one score column
data['test_score_original'] = data['test_score']
data['test_score'] = data['mean_bootstrap']
data['test_lower_ci'] = data['lower_ci']
data['test_upper_ci'] = data['upper_ci']
data['val_score'] = data['val_mean_bootstrap']

data['model_modality'] = data['mul_uni']
data['data_alignment'] = data['alignment']

#insert a new second column in data called model_id that combines model + train_type
data.insert(0, 'model_id', data['model'] + '-' + data['train_type'])

#insert a new second column in data called electrode_id that combines electrode and subject
data.insert(1, 'electrode_id', data['subject'] + '-' + data['electrode'])

#keep only the columns we'll need in the analysis
data = data[['model_id','model','train_type','model_modality',
             'data_alignment','electrode_id','times',
             'val_score','val_lower_ci','val_upper_ci',
             'test_score', 'test_lower_ci', 'test_upper_ci', 'test_score_original']]

#drop all duplicate columns from data
data = data.drop_duplicates()

In [ ]:
def relabel_modality(row):
    modality = row['model_type']
    if modality == 'multimodal' and row['train_type'] == 'randomized':
        if not any(x in row['model'] for x in ['lxmert', 'visual_bert']):
            if 'language' in row['model']:
                modality = 'unimodal_language'
            if 'vision' in row['model']:
                modality = 'unimodal_vision'
            
    return modality

In [ ]:
data.groupby(['data_alignment'])['electrode_id'].nunique()

In [ ]:
data.groupby(['data_alignment'])['model_id'].nunique()

In [ ]:
group_summary_stats(data, ['data_alignment'], 'times')

In [ ]:
model_data = data[['model_id','model','train_type','model_modality']]
model_data = model_data.drop_duplicates().reset_index(drop=True)
#model_data['model_type'] = model_data.apply(lambda row: relabel_modality(row), axis = 1)

In [ ]:
electrode_id = data['electrode_id'].unique()[5]
nest1 = data[data['electrode_id'] == electrode_id]
nest2 = nest1[nest1['data_alignment'] == 'language']
nest3 = nest2[nest2['model_id'] == 'albef-trained']

In [ ]:
group_vars = ['model_id', 'electrode_id', 'data_alignment']
summary = {'val':  group_summary_stats(data, group_vars, 'val_score'),
           'test': group_summary_stats(data, group_vars, 'test_score_original')}

In [ ]:
summary['val']['val_score_mean'].min(), summary['val']['val_score_mean'].max()

In [ ]:
summary['test'].groupby(['model_id','data_alignment'])['electrode_id'].nunique().reset_index();

In [ ]:
#loop through each model_id and each electrode_id, bootstrap the mean score across times and save the results in a new dataframe associated with model ranking
model_ranklists = make_ranklists(data)

### Comparisons

In [ ]:
default_winners, comparison_data  = model_comparisons(data, model_ranklists)
comparison_data 

In [ ]:
default_winners.groupby(['comparison'])['electrode_id'].nunique().reset_index()

In [ ]:
from statsmodels.stats.multitest import multipletests

analysis_data = comparison_data.copy()

comparison_list = analysis_data.comparison.unique()

analysis_dflist = []
for comparison in comparison_list:
    cur_data = analysis_data[analysis_data['comparison'] == comparison].copy()
    
    cur_data['p_value_adj'] = (multipletests(cur_data['p_value'].values,
                                             method = 'fdr_bh')[1])
    
    electrodes_remaining = cur_data[cur_data['p_value_adj'] < 0.05].shape[0]
    print(f'electrodes passing comparison {comparison}: {electrodes_remaining}')

    cur_data['significant'] = cur_data['p_value_adj'] < 0.05
    
    analysis_dflist.append(cur_data)
    
analysis_data = pd.concat(analysis_dflist)

In [ ]:
analysis_data.to_parquet('../final-results/comparisons.parquet.gzip', compression = 'gzip')

In [ ]:
analysis_data

In [ ]:
analysis_data_parsed = (analysis_data.query('p_value_adj < 0.05')
                        .query('model1_nobs >= 10')
                        .query('model2_nobs >= 10')).copy()

analysis_data_parsed

comparison_list = ['rank1-beats-rank2', 'trained-beats-random','random-beats-trained', 
                   'multi-beats-unimodal', 'slip-beats-simclr']

#for comparison in analysis_data_parsed['comparison'].unique():
print('---Bootstrapped Comparisons---')
for comparison in comparison_list:
    cur_data = analysis_data_parsed[analysis_data_parsed['comparison'] == comparison]
    print(comparison, ':', cur_data['electrode_id'].nunique())
    
for comparison in comparison_list:
    cur_data = analysis_data_parsed[analysis_data_parsed['comparison'] == comparison]
    if comparison == 'multi-beats-unimodal':
        cur_data = cur_data.groupby('alignment')['electrode_id'].nunique().reset_index().query('electrode_id > 1')
        comparison = 'multi-beats-unimodal (both alignments)'
        print(comparison, ':', cur_data['electrode_id'].nunique())
        
    
    if comparison == 'slip-beats-simclr':
        cur_data = cur_data.groupby('alignment')['electrode_id'].nunique().reset_index().query('electrode_id > 1')
        comparison = 'slip-beats-simclr (both alignments)'
        print(comparison, ':', cur_data['electrode_id'].nunique())
        
    
from os import linesep
print(linesep)
    
print('---Default Winners---')
for comparison in default_winners.comparison.unique():
    cur_data = default_winners[default_winners['comparison'] == comparison]
    
    if comparison == 'multimodal-beats-unimodal':
        comparison = 'multi-beats-unimodal'
        
    if comparison == 'slipclr-beats-slip':
        comparison = 'slip-beats-simclr'
    
    print(comparison, cur_data['electrode_id'].nunique())
    
for comparison in default_winners.comparison.unique():
    
    cur_data = default_winners[default_winners['comparison'] == comparison]
    if comparison == 'multimodal-beats-unimodal':
        cur_data = cur_data.groupby('alignment')['electrode_id'].nunique().reset_index().query('electrode_id > 1')
        comparison = 'multi-beats-unimodal (both alignments)'
        print(comparison, ':', cur_data['electrode_id'].nunique())
        
    if comparison == 'slip-vision-beats-simclr':
        cur_data = cur_data.groupby('alignment')['electrode_id'].nunique().reset_index().query('electrode_id > 1')
        comparison = 'slip-beats-simclr (both alignments)'
        print(comparison, ':', cur_data['electrode_id'].nunique())

In [ ]:
analysis_data_parsed.to_parquet('../final-results/parsed_comparison_data.parquet.gzip')

In [ ]:
summary['test'].query('count >= 10')#.to_csv('scoring_data.csv', index = None)

In [ ]:
summary['test'].query('count >= 0')

In [ ]:
(group_summary_stats(summary['test'].query('count >= 0'), ['model_id', 'data_alignment'], 'test_score_original_mean', use_value_name = False)
 .round(4).sort_values('mean', ascending=False))#.to_csv('trained_beats_random.csv', index = None)

In [ ]:
train_tests =summary['test'].query('count >= 0').merge(model_data, on = 'model_id', how = 'left')
train_tests[train_tests['model_id'].str.contains('simcse')]

In [ ]:
(group_summary_stats(train_tests[train_tests['model_id'].str.contains('simcse')], ['model_id', 'data_alignment'], 
                     'test_score_original_mean', use_value_name = False)
 .round(4).sort_values(['data_alignment','mean'], ascending=False))#.to_csv('trained_beats_random.csv', index = None)

In [ ]:
(group_summary_stats(train_tests, ['train_type', 'data_alignment'], 'test_score_original_mean', use_value_name = False)
 .round(4).sort_values('mean', ascending=False))